# Mifare Classic Attack

Attack Phases:
1. Load the raw signal of the communication.
1. Analyze the raw signal and demodulate the communication between the Reader and a MIFARE Classic.
1. Associate every detected message to the most likely MIFARE Classic command.
1. Use the keys generated by CRYPTO1 cypher during the communication to decrypt the result of a READ.

📝*Note:* Before running the notebook it's necessary to capture the raw signals using ***GNURadio*** and the ***mfclassic_apdu_get_data*** script.

In [ ]:
# Import
from common_imports import *

# Useful in Windows OS
import warnings
warnings.filterwarnings('ignore')

### 1️⃣ *Loading Data and show the entire communication.*

In [ ]:
# PATHS
DATA_PATH    = r'../data/classic/'
RESULTS_PATH = r'../data/results/'

# ID of the attacked MIFARE Classic
MIFARE_CLASSIC_ID   = "7e a2 42 3d a3"

# Data previously written into the owned MIFARE Classic
MIFARE_CLASSIC_DATA = "11 22 33 44 55 66 77 88 99 77 77 77 77 77 77 77"

# Specify one of the blocking card contained in BLOCKING_CARD_LIST
BLOCKING_CARD_LIST = ['BC1','BC2','BC2','BC3','BC4','BC5','BC6','BC7','BC8','BC9','BC10','BC11','BC12','BC13','BC14']
ATTACKED_BLOCKING_CARD = 'BC1'


In [ ]:
# Paths
dataPath = DATA_PATH + 'classic_{}.raw'.format(ATTACKED_BLOCKING_CARD)
expected_file = libnfc_file = DATA_PATH + 'expectedTxt/classic_{}.txt'.format(ATTACKED_BLOCKING_CARD)

# Load data
data = load_mag(dataPath)

# Print the signal
plt.figure(figsize=(50, 6), color="#47B5FF")
plt.plot(data)
plt.show()

### 2️⃣ *Demodulate all the messages in the communication.*

In [ ]:
# Load everything into a NfcSignal class
start = time()
s = NfcSignal(data, 
      expected_file = expected_file, 
      libnfc_file = libnfc_file,
      attack_mode = 0, 
      mean_samples = 0, 
      message_batch_size = 8)
end = time()
print(f"init duration {end-start}")

In [ ]:
# Start Demodulation
s.start_demodulation()

Print demodulation stats.

**OR**

Save demodulation stats into a CSV file.

In [ ]:
# If you want to store demodulation stats
#stats_file = '../data/csv/classic_clean_demodulation_stats.csv'
#s.demodulation_stats().to_csv(stats_file)

# If you don't need to store demodulation stats
s.demodulation_stats()

#### Create a DF with each message and its HEX representation

In [ ]:
# Empty list to create the DF
msgList = []

# Iterate through all the messages detected using NfcSignal class
for message_start, message_end in zip(s.message_start, s.message_end):
    for m_s, m_e, m_d, m_t in zip(message_start, message_end, s.message_detected, s.message_type):
        if m_d:
            # Retrieve the msg
            message = s.signal_normalized[m_s:m_e]
            
            # Perform Demodulation
            hex_message = NfcSignal.perform_demodulation(
                message, 
                device_flag = m_t, 
                show_plots = False,
                hide_demodulated_data = True)
            hex_message = hex_message.rstrip()

            # Compute the number of bytes of each msg
            hex_message_num_bytes = int(len(hex_message.replace(" ", ""))/2)

            # Save results of demodulation
            msgList.append([str(m_t),hex_message,hex_message_num_bytes])

# Create the DF from the list
msgDf = pd.DataFrame(msgList,columns=['msgType','msgHex','numBytes'])

### 3️⃣ *Try to map every HEX representation into a MIFARE Classic command.*
Using the **MIFARE Classic Manual(Chapter 9)** map every HEX msg into the most likely MIFARE Classic command.

In [ ]:
# Add some useful columns to the DF
msgDf["cmd"] = " "
msgDf["cmdHex"] = " "

In [ ]:
#Define a list of possible cmds according to Table 9 of Mifare Classic Manual
cmdList = [
    ["26","REQA",1],
    ["52","WUPA",1],
    ["93 20","Anticollision CL1",2],
    ["93 70","Select CL1",9],
    ["95 20","Anticollision CL2",2],
    ["95 70","Select CL2",9],
    ["50 00","Halt",4],
    ["60","Authentication with Key A",4],
    ["61","Authentication with Key B",4],    
    ["40","Personalize UID Usage",4],
    ["43","Set MOD_TYPE",4],
    ["30","Mifare Read",4],
    ["a0","Mifare Write",4],
    ["c0","Mifare Decrement",4],
    ["c1","Mifare Increment",4],
    ["c2","Mifare Restore",4],
    ["b0","Mifare Transfer",4],
    ["04 00","ATQA",2],
    ["08 b6 dd","The card is a Mifare 1k",3],
    [MIFARE_CLASSIC_ID,"ID of my Mifare Classic",5]
]

cmdDf = pd.DataFrame(cmdList,columns=['cmdHex','cmd','expectedNumBytes'])

In [ ]:
# Iterate through all the messages
for i in range(len(msgDf)):

    # Select only the possible cmd from the cmd DF according to the expected num of bytes
    tmpDf = cmdDf.loc[cmdDf['expectedNumBytes'] == msgDf.loc[i,'numBytes']]
    tmpDf.index = range(len(tmpDf))

    # Find the most likely cmd
    for j in range(len(tmpDf)):
        if(msgDf.loc[i,'msgHex'].startswith(tmpDf.loc[j,'cmdHex']) and msgDf.loc[i,'numBytes'] == tmpDf.loc[j,'expectedNumBytes']):
            msgDf.loc[i,'cmd'] = tmpDf.loc[j,"cmd"]
            msgDf.loc[i,'cmdHex'] = tmpDf.loc[j,'cmdHex']

Save the detected commands into a CSV file.

**OR**

Print the detected commands on the screen.

In [ ]:
# To save the whole communication into a CSV file
# msgDf.to_csv('../data/csv/classic_clean.csv')

# To show the whole communication
msgDf

### 4️⃣ *Use the keys generated by CRYPTO1 cypher to decrypt the result of a READ.*

The file containing the decryption keys is the one generated by the ***mfclassic_apdu_get_data*** script during the acquistion phase.

In [ ]:
# Get possible Card Data Messages (the one composed by 18 bytes)
encryptedCardDataList = msgDf.loc[msgDf['numBytes'] == 18]['msgHex'].tolist()

# Get the decryption keys
decryptionKeysFilePath = DATA_PATH + 'decryptionKeys/classic_{}_decryption_keys.txt'.format(ATTACKED_BLOCKING_CARD)
decryptionKeysList = open(decryptionKeysFilePath , "r").read().split("\n")[:-1]

# Decrypted Card Data
decryptedCardData = []

# Try to decrypt
for m in encryptedCardDataList:
    print("[⛏️ Trying to decrypt]: {}\n".format(m))

    # Try to use all keys
    for k in decryptionKeysList:
        
        key_bytes = [int(val, 16) for val in k.split()]
        encrypted_bytes = [int(val, 16) for val in m[:-2].split()]

        decrypted = []

        # XORing
        for i in range(len(key_bytes)):
            decrypted.append("{:02x}".format(key_bytes[i] ^ encrypted_bytes[i]))
        
        decrypted = " ".join(decrypted)
        if decrypted == MIFARE_CLASSIC_DATA:
            print("\t[✅ FOUND]    : {}".format(decrypted))
            print("\t[🔑 USING KEY]: {} \n".format(k))
    